# Model Evaluation and Data Augmentation

Now that we have trained our model, let's see how good the model is. There are several approaches for evaluating model training and classification performance such as training curves, AUROC, and F1 score.

### Setup drive

Run the following cell to mount your Drive onto Colab. Go to the given URL and once you login and copy and paste the authorization code, you should see "drive" pop up in the files tab on the left.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Click the little triangle next to "drive" and navigate to the "AI4All Chest X-Ray Project" folder. Hover over the folder and click the 3 dots that appear on the right. Select "copy path" and replace `PASTE PATH HERE` with the path to your folder.

In [ ]:
cd "/content/drive/My Drive/AI4All Chest X-Ray Project"

/content/drive/.shortcut-targets-by-id/1iJKbtzLay6C-5OfpVHhbe1nQeNGyhFWO/AI4All Chest X-Ray Project


### Import necessary libraries

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split

import torchvision
from torchvision import datasets, transforms

from utils.plotting import imshow_dataset

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Setup paths

In [ ]:
path_to_dataset = os.path.join('data')

path_to_images = os.path.join(path_to_dataset, 'images')

metadata = pd.read_csv(os.path.join(path_to_dataset, 'metadata_train.csv'))

### Define helper functions

We worked on these functions in the last notebook. They are defined here in case you want to take a look at how they are implemented. I will also provide them in the helper library so that they can be imported for future notebooks (without copying the code over and over).

**Define functions for splitting dataset**

In [ ]:
def get_train_val_sizes(dataset, train_ratio):
  '''Calculate train and validation sizes based on dataset size and ratio of 
  data to keep in training set'''

  train_size = int(train_ratio * len(dataset))
  val_size = len(dataset) - train_size

  return train_size, val_size

In [ ]:
def get_train_val_sets(dataset, train_ratio, batch_size):
  '''Randomly splits data into training and validation sets based on specified ratio'''

  # calculate training and validation set sizes
  train_size, val_size = get_train_val_sizes(dataset, train_ratio)

  # split dataset into two training set and validation set
  train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

  # create dataloaders
  train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)
  val_loader = DataLoader(dataset=val_dataset, shuffle=False, batch_size=len(val_dataset))

  return train_loader, val_loader

**Define functions for training neural network**

In [ ]:
def calc_accuracy(predicted, actual):
    '''Given predicted and actual labels, calculate accuracy'''
    accuracy = torch.mean((predicted == actual).float())
    return accuracy

In [ ]:
def predict(model, inputs):
    '''Return class label with highest probability as the model prediction'''
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    return predicted

In [ ]:
def train(model, train_loader, val_loader, epoch_num=1, lr=0.001):
    '''Trains a neural network

    Args:
        model: torch model
        train_loader (DataLoader): training data
        val_loader (DataLoader): validation data
        epoch_num (int): number of epochs
        lr (float): learning rate

    Return:
        None (plots loss over iterations)
    '''
    
    # define loss and optimization functions
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # load validation set
    inputs_val, targets_val = next(iter(val_loader))
    
    # create variables for tracking loss and accuracy values
    train_accuray_log = []
    val_accuracy_log = []
    loss_log = []

    for epoch in range(epoch_num):

        for batch_num, data in enumerate(train_loader):

            inputs, targets = data

            # set parameter gradients to zero
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)

            # calculate loss and update weights
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

            # calculate accuracy
            _, predicted = torch.max(outputs, 1)
            train_accuracy = calc_accuracy(predicted, targets)
            
            # calculate validation accuracy

            predicted = predict(model, inputs_val)
            val_accuracy = calc_accuracy(predicted, targets_val)

            # record loss and accuracy values
            loss_log.append(loss)
            train_accuray_log.append(train_accuracy.item())
            val_accuracy_log.append(val_accuracy.item())

            # print loss
            print('[epoch {} batch {}] Loss: {:.3f} Train accuracy: {:.3f} Val accuracy: {:.3f}'
                  .format(epoch, batch_num, loss, train_accuracy, val_accuracy))


    print('Finished Training')
    return train_accuray_log, val_accuracy_log, loss_log

### Train NN model

This is the whole pipeline so far! We've gone through preprocessing, splitting the data into training and validation sets, setting up a neural network, and training the model. (Whew!)

Play around with modifying parameters and inputs in this section to change your neural network!

**Preprocess data**


In [ ]:
DATA_MEAN = 0.544
DATA_STD = 0.237

# smaller image size for quicker training
RESIZE_SIZE = 50
CROP_SIZE = 50 

IM_SIZE = CROP_SIZE

# transformations
data_transforms = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize(IM_SIZE),
        transforms.CenterCrop(IM_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=DATA_MEAN, std=DATA_STD)])

dataset = datasets.ImageFolder(path_to_images, transform=data_transforms)

**Split the data into training and validation set**



In [ ]:
train_ratio = 0.75
batch_size = 64
train_loader, val_loader = get_train_val_sets(dataset, train_ratio, batch_size)

**Setup fully connected network**



In [ ]:
input_size = IM_SIZE * IM_SIZE

# define network
model = nn.Sequential(nn.Flatten(),
                      nn.Linear(input_size, 64),
                      nn.ReLU(),
                      nn.Linear(64, 16),
                      nn.ReLU(),
                      nn.Linear(16, 2),
                      nn.LogSoftmax(dim=1))

print(model)

Sequential(
  (0): Flatten()
  (1): Linear(in_features=2500, out_features=64, bias=True)
  (2): ReLU()
  (3): Linear(in_features=64, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=2, bias=True)
  (6): LogSoftmax()
)


**Train network**

Note: if the model is running too slow, try defining a smaller model for speed

In [ ]:
train_accuray_log, val_accuracy_log, loss_log = train(model, train_loader, val_loader)

## Evaluate model performance

It's hard to look at the printed output to evaluate the model performance. Let's consider a few evaluation approaches using plots and metrics.

### Training curves

Plotting the model performance over training iterations (batches / epochs) is helpful for assessing how the model training is going. Loss (error) and accuracy (correctness) are two useful indicators to plot.

**Loss curve**

The loss curve tracks the loss (error) over iterations (training).  We used the negative log-likelihood loss (NLLLoss) in the model. In combination with log softmax activation in the final layer, this produces the cross entropy loss. You have already calculated the loss log (`loss_log`) over iterations in the train function - just need to plot it! 

Note: In addition to the training loss, you can modify the `train` function to record the validation loss and it plot to compare with the training loss.



In [ ]:
# EXERCISE: Plot the loss curve over iterations (batches). The plot should show 
# loss on the y-axis and iterations on the x-axis.
# Tip: plt.plot()

def plot_loss_curve(loss):

  ### WRITE CODE HERE ###


In [ ]:
plot_loss_curve(loss_log)

**Questions:** Based on the loss curve, is the model converging? Has it converged? How stable is the training? 

Try changing the epoch number, batch size, learning rate, or other parameters and check how the loss curve changes.

**Accuracy curves**

Similarly, plot the training and validation accuracy curves. Comparing the model performance on training and validation sets allows us to assess whether the model is overfitting or underfitting.

In [ ]:
# EXERCISE: Plot the training and validation accuracy curves on the same plot.

def plot_training_curve(train, val):

    ### WRITE CODE HERE ###


In [ ]:
plot_training_curve(train_accuray_log, val_accuracy_log)

**Question:** Comparing the training and validation curves, is the model overfitting or underfitting?

### Evaluation metrics

**Confusion matrix**

The confusion matrix summarizes the predictions across their actual class labels. [Read more here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

In [ ]:
def plot_confusion_matrix(confusion_mat):
  '''Plots the confusion matrix

  Args:
    confusion_mat: confusion matrix
  '''
  cm_df = pd.DataFrame(
      cm, 
      index = [idx for idx in ['COVID-19', 'No Finding']],
      columns = [col for col in ['COVID-19 (pred)', 'No Finding (pred)']])
  plt.figure(figsize = (8,6))
  sns.heatmap(cm_df, annot=True)
  plt.show()

In [ ]:
# EXERCISE: Calculate the confusion matrix for the validation set and plot it.
# Tip: use the imported confusion_matrix function to calculate the matrix and 
# the plot_confusion_matrix function above to plot the matrix.

from sklearn.metrics import confusion_matrix

### WRITE CODE HERE ###


**Question**: What do each cell of the matrix represent? What are the true positive rate and the false positive rate?

**AUROC**

There are two common ways to combine the information in a confusion matrix into a single metric for evaluating model performance: AUROC and F1 score

The ROC plots the true postive rate versus the false positive rate ([read more here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)). AUROC is the area under the curve. An AUROC of 1 indicates good model discrimination between the classes whereas an AUROC of 0.5 indicates poor discrimination. 

In [ ]:
def flip_labels(labels):
  '''Swaps Covid (0 to 1) and No Finding (1 to 0) labels'''
  return 1 - labels

In [ ]:
# EXERCISE?

from sklearn.metrics import roc_curve, auc

# calculate roc curve and auroc
outputs = model(inputs_val)
log_covid_prob = torch.index_select(outputs, 1, torch.tensor([0])).detach()

covid_prob = torch.exp(log_covid_prob)

fpr, tpr, _ = roc_curve(flip_labels(targets_val), covid_prob)
auroc = auc(fpr, tpr)

# plot roc curve
plt.plot(fpr, tpr, color='red')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title(f'ROC Curve: area = {auroc:.3f}')


**Precision, recall, F1 score**

Another way to measure model performance is using precision and recall. These are also calculated using the the values in the confusion matrix. The F1 score combines precision and recall into a single metric. ([Read more here](https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py))

**Question:** How do precision and recall differ from true positive rate or false positive rate?

In [ ]:
# EXERCISE: Use the important methods to calculate precision, recall, and f1 score

from sklearn.metrics import precision_score, recall_score, f1_score

targets_val = flip_labels(targets_val)
predicted = flip_labels(predicted)

precision_score = # CODE HERE #
recall_score =  # CODE HERE #
f1_score =  # CODE HERE #

print(f'Precision: {precision_score:.3f}')
print(f'Recall: {recall_score:.3f}')
print(f'F1 Score: {f1_score:.3f}')

**Error Analysis** 

Metrics are one way to assess a model. We can often gain more insight by examining the predictions directly. By analyzing when (for which cases) the model predicts incorrectly, we can generate additional hypotheses on how we should train the model for better performance.

Some ideas to consider
- Duplicated patients in training and validation sets
- Metadata information: views, location, sex/age
- Severity

This a potential follow-up idea if you are interested!

### Apply evaluation

Apply the evaluation methods above to your model. Play around with tweaking model parameters for better performance.

In [ ]:
# EXERCISE



### Data Augmentation (Bonus)

If the model does not have enough training examples or is not generalizing well (overfitting), data augmentation is one strategy to increase the training data size and to broaden the range of training signal. We can use data transforms to define augmentation the preprocessing step. 

**Agumentation transforms**

Apply transforms that could improve the generalizability of the model by adding realistic diversity to the dataset. Consider which transforms to use. Some transforms may actually increase the bias in the data and produce worse performance.


In [ ]:
# EXERCISE: Define augmentation transforms

augmentation_transforms = # CODE HERE #


In [ ]:
imshow_dataset(dataset, rand=True)

**Add data augmentation transformations to preprocessing**

Make sure resize and center crop are appropriately resized so that the region of interest (lung area) is not cropped out and the image still covers the full frame.




In [ ]:
# EXERCISE: Modify transforms as needed

data_transforms = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize(256),
            *augmentation_transforms,
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[DATA_MEAN], std=[DATA_STD])])

dataset = datasets.ImageFolder(path_to_images, transform=data_transforms)

In [ ]:
imshow_dataset(dataset, rand=True)

**Retrain network**

Apply transforms with augmentation to the pipeline and retrain the network. How does it improve performance?

Note: The augmentation transforms should only be added to training set. (Why?)

In [ ]:
# EXERCISE

